# QUEUES

## NOTE: this needs to be run in docker because graphviz is a pain to install in OS X

Queues are the backbone of numerous algorithms found in games, artificial intelligence, satellite navigation, and task scheduling. They’re among the top abstract data types that computer science students learn early in their education. At the same time, software engineers often leverage higher-level message queues to achieve better scalability of a microservice architecture. Plus, using queues in Python is simply fun!

Python provides a few built-in flavors of queues that you’ll see in action in this tutorial. You’re also going to get a quick primer on the theory of queues and their types. Finally, you’ll take a look at some external libraries for connecting to popular message brokers available on major cloud platform providers.

In this tutorial, you’ll learn how to:

    Differentiate between various types of queues
    Implement the queue data type in Python
    Solve practical problems by applying the right queue
    Use Python’s thread-safe, asynchronous, and interprocess queues
    Integrate Python with distributed message queue brokers through libraries


## 004. THIS IS ABOUT...

https://realpython.com/queue-in-python/

In [1]:
%load_ext nb_mypy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # type: ignore

Version 1.0.5


In [2]:
from collections import deque
from heapq import heappop, heappush
import networkx as nx
from typing import Any, Callable, Deque, Generic, NamedTuple, TypeVar
from enum import IntEnum

### 001.001 Building a Queue Data Type

Because you want your custom FIFO queue to support at least the enqueue and dequeue operations, go ahead and write a bare-bones Queue class that’ll delegate those two operations to `deque``


1. Create a BasicQueue FIFO queue class which
    1. starts with an empty queue - make sure the elements are stored in `_elements` and are all of the same type, and the class knows about it
    1. has an `enqueue` method to add items to the queue
    1. has a `deque` method which gets it off it
    1. uncomment the assertions to prove they work
1. Create a slightly more complex Queue class which
    1. can take a list of elements
    1. enqueue and dequeue as before
    1. allows `len(an_instance)` to return the length
    1. allows `for i in an_instance...` to iterate through items
    1. uncomment the assertions to prove they work


In [3]:


T = TypeVar('T')

1
# class BasicQueue...something T...:
#     def __init__(self):
#         """1.1"""

#     def enqueue(self, element: T) -> None:
#         """1.2"""

#     def dequeue(self) -> T:
#         """1.3"""

# 1.4
# basic_fifo = BasicQueue()
# basic_fifo.enqueue(1)
# basic_fifo.enqueue(2)
# basic_fifo.enqueue(3)


# assert basic_fifo.dequeue() == 1
# assert basic_fifo.dequeue() == 2
# assert basic_fifo.dequeue() == 3

2
# class Queue...something T...:
#     def __init__(self, extend_to, any, number):
#         """2.1"""

#     def enqueue(self, element: T) -> None:
#         """2.2"""

#     # def dequeue(self) -> T:
#     #     """2.2"""

#     2.3
#     """length"""

#     2.4
#     """for..."""

2.5
# fifo = Queue(1, 2, 3)
# assert len(fifo) == 3
# assert [1,2,3] == [el for el in fifo]
# assert len(fifo) == 0

# fifo.enqueue(1)
# fifo.enqueue(2)
# fifo.enqueue(3)

# assert fifo.dequeue() == 1
# assert fifo.dequeue() == 2
# assert fifo.dequeue() == 3

# solution


class BasicQueue(Generic[T]):
    def __init__(self):
        self._elements:Deque[T] = deque()

    def enqueue(self, element:T) -> None:
        self._elements.append(element)

    def dequeue(self) -> T:
        return self._elements.popleft()

basic_fifo = BasicQueue[int]()
basic_fifo.enqueue(1)
basic_fifo.enqueue(2)
basic_fifo.enqueue(3)

assert basic_fifo.dequeue() == 1
assert basic_fifo.dequeue() == 2
assert basic_fifo.dequeue() == 3

class Queue(Generic[T]):
    def __init__(self, *elements: T):
        self._elements: Deque[T] = deque(elements)

    def enqueue(self, element:T) -> None:
        self._elements.append(element)

    def dequeue(self) -> T:
        return self._elements.popleft()

    def __len__(self) -> int:
        return len(self._elements)

    def __iter__(self):
        while len(self) > 0:
            yield self.dequeue()


2.5
fifo = Queue[int](1, 2, 3)
assert len(fifo) == 3
assert [1,2,3] == [el for el in fifo]
assert len(fifo) == 0

fifo.enqueue(1)
fifo.enqueue(2)
fifo.enqueue(3)

assert fifo.dequeue() == 1
assert fifo.dequeue() == 2
assert fifo.dequeue() == 3

<cell>63: note: By default the bodies of untyped functions are not checked, consider using --check-untyped-defs  [annotation-unchecked]


1

2

2.5

2.5

### 001.002 Building a Stack Data Type

Building a stack data type is considerably more straightforward because you’ve already done the bulk of the hard work. Since most of the implementation will remain the same, you can extend your Queue class using inheritance and override the .dequeue() method to remove elements from the top of the stack

(NOTE: this doesn't imply that conceptually Stack extends Queue, it's simply for convenience)

1. Create a Stack class
    1. It inherits from Queue
    1. It overwrites the dequeue method to remove elements from the top
    1. uncomment assertions

In [4]:
1
# class Stack(Queue ...something T...):
#     def dequeue(self):
#         """1.1"""

# lifo = Stack(1, 2, 3)
# assert len(lifo) == 3
# assert [3,2,1] == [el for el in lifo]
# assert len(lifo) == 0

# solution

class Stack(Queue, Generic[T]):

    def dequeue(self) -> T:
        return self._elements.pop()

lifo = Stack[int](1, 2, 3)
assert len(lifo) == 3
assert [3,2,1] == [el for el in lifo]
assert len(lifo) == 0

1

### 001.003 Building a Priority Queue

Fortunately, you can be smart about keeping the elements sorted in a priority queue by using a heap data structure under the hood. It provides the most efficient implementation

Python has the heapq module, which conveniently provides a few functions that can turn a regular list into a heap and manipulate it efficiently.

1. Create a PriorityQueue class
    1. Initialised with 
        1. It stores elements in a simple list
        1. It uses a standard generator from itertools to keep track of next item
        1. ...which needs to be imported
    1. enqueue_with_priority leverages Python’s tuple comparison, which takes into account the tuple’s components, looking from left to right until the outcome is known
        1. Elements contain the priority first...
            1. Don't forget to type annotate element
            1. Q Note that heapq is a min-heap, i.e..... How does that change the way priority is encoded? 
            1. We would like to maintain the chronological ordering of queue items if they have the same priority. That's what the counter is all about
            1. Finally the value
        1. Then the element is finally pushed into the heap
    1. Dequeue only fetches the value from the helement, letting heapq do the work
        

In [5]:
class Priority(IntEnum):
    CRITICAL = 3
    IMPORTANT = 2
    NEUTRAL = 1

1
# class PriorityQueue:
#     def __init__(self):
#         """1.1.1"""
#         """1.1.2"""

#     def enqueue_with_priority(self, priority:Priority, value:str) -> None:
#         """1.2.1"""
#         element = ...
#             # 1.2.1.1 Q: Note that heapq is a min-heap, i.e....
#             # 1.2.1.2
#             # 1.2.1.3
#             # 1.2.1.4
#         # 1.2.2

#     def dequeue(self) -> str:
#         """1.3"""

# 2
# messages = PriorityQueue()
# messages.enqueue_with_priority(Priority.IMPORTANT, "Windshield wipers turned on")
# messages.enqueue_with_priority(Priority.NEUTRAL, "Radio station tuned in")
# messages.enqueue_with_priority(Priority.CRITICAL, "Brake pedal depressed")
# messages.enqueue_with_priority(Priority.IMPORTANT, "Hazard lights turned on")

# assert messages.dequeue() == 'Brake pedal depressed'

# solution
#1.1.3
from itertools import count

1
class PriorityQueue():
    def __init__(self):
        self._elements: list[tuple[int, int, str]] = []
        self._counter = count()

    def enqueue_with_priority(self, priority:Priority, value:str) -> None:
        # 1.2.1.1 A: the very first element in always the smallest, so not what we want
        element: tuple[int, int, str] = (
            # 1.2.1.2 that's why priority is -ve
            -priority,
            # 1.2.2.3
            next(self._counter),
            # 1.2.2.4
            value)
        # 1.2.2
        heappush(self._elements, element)

    def dequeue(self) -> str:
        return heappop(self._elements)[-1]

2
messages = PriorityQueue()
messages.enqueue_with_priority(Priority.IMPORTANT, "Windshield wipers turned on")
messages.enqueue_with_priority(Priority.NEUTRAL, "Radio station tuned in")
messages.enqueue_with_priority(Priority.CRITICAL, "Brake pedal depressed")
messages.enqueue_with_priority(Priority.IMPORTANT, "Hazard lights turned on")

assert messages.dequeue() == 'Brake pedal depressed'




<cell>40: note: By default the bodies of untyped functions are not checked, consider using --check-untyped-defs  [annotation-unchecked]


1

1

2

### 001.004 Refactoring with mixins

`len` and `__iter__` are common to all 3 classes so far, and can be espressed as mixins. Assume `_elements` is common to all

1. Extract `__len__` and `__iter__` from earlier Queue class
1. Extend all classes created so far (Queue, Stack, PriorityQueue) with it



In [6]:
1
class IterableMixin:
    def __len__(self) -> int:
        return len(self._elements) # type: ignore

    def __iter__(self):
        while len(self) > 0:
            yield self.dequeue()

# class Queue(IterableMixin):
#     ...

# class Stack(Queue):
#     ...

# class PriorityQueue(IterableMixin):
#     ...

# fifo = Queue(1, 2, 3)
# assert len(fifo) == 3
# assert [1,2,3] == [el for el in fifo]
# assert len(fifo) == 0

# lifo = Stack(1, 2, 3)
# assert len(lifo) == 3
# assert [3,2,1] == [el for el in lifo]
# assert len(fifo) == 0

# messages = PriorityQueue()
# messages.enqueue_with_priority(Priority.IMPORTANT, "Windshield wipers turned on")
# messages.enqueue_with_priority(Priority.NEUTRAL, "Radio station tuned in")
# messages.enqueue_with_priority(Priority.CRITICAL, "Brake pedal depressed")
# messages.enqueue_with_priority(Priority.IMPORTANT, "Hazard lights turned on")

# assert messages.dequeue() == 'Brake pedal depressed'

# solution

class Queue(IterableMixin, Generic[T]):
    def __init__(self, *elements: T):
        self._elements = deque(elements)

    def enqueue(self, element:T):
        self._elements.append(element)

    def dequeue(self) -> T:
        return self._elements.popleft()

class Stack(Queue, Generic[T]):
    def dequeue(self) -> T:
        return self._elements.pop()

class PriorityQueue(IterableMixin):
    def __init__(self):
        self._elements: list[tuple[int, int, str]] = []
        self._counter = count()

    def enqueue_with_priority(self, priority:Priority, value:str) -> None:
        # 1.2.1.1 A: the very first element in always the smallest, so not what we want
        element: tuple[int, int, str] = (
            # 1.2.1.2 that's why priority is -ve
            -priority,
            # 1.2.2.3
            next(self._counter),
            # 1.2.2.4
            value)
        # 1.2.2
        heappush(self._elements, element)

    def dequeue(self) -> str:
        return heappop(self._elements)[-1]



fifo = Queue[int](1, 2, 3)
assert len(fifo) == 3
assert [1,2,3] == [el for el in fifo]
assert len(fifo) == 0

lifo = Stack[int](1, 2, 3)
assert len(lifo) == 3
assert [3,2,1] == [el for el in lifo]
assert len(fifo) == 0

messages = PriorityQueue()
messages.enqueue_with_priority(Priority.IMPORTANT, "Windshield wipers turned on")
messages.enqueue_with_priority(Priority.NEUTRAL, "Radio station tuned in")
messages.enqueue_with_priority(Priority.CRITICAL, "Brake pedal depressed")
messages.enqueue_with_priority(Priority.IMPORTANT, "Hazard lights turned on")

assert messages.dequeue() == 'Brake pedal depressed'




<cell>55: note: By default the bodies of untyped functions are not checked, consider using --check-untyped-defs  [annotation-unchecked]


1

### 001.005 Exploring networkx, graphviz and DOT

The DOT graph description language defines a file format that is most often used in the context of graph visualization with Graphviz. NetworkX provides an interface to Graphviz via pygraphviz, implemented in nx_agraph. If pygraphviz is installed, nx_agraph can be used to read and write files in DOT format.


1. Read `roadmap_file` into var `graph` as a networkx data structure, using the `pygraphviz` extension
    1. networkx graph nodes are textual identifiers that can optionally have an associated dictionary of attributes. Print the one for "london" to see what it looks like
2. Instead of a dict we want a data structure which is hashable but less["awkward"] to work with. Create a class City
    1. the `from_dict` method takes the dict from 1.1 and returns a City instance
    1. note that year could be not found
    1. make sure you cast all non-string types
3. Create a couple of helpers function
    1. `as_cities_map` should cycle through `graph.nodes` (from 1.) and create a similar dict but with City (from 2.) as value
        1. TIP: you need to pass `data=True` when iterating though nodes, so that you get a dict
    1. create `cities_map` with it
    1. `as_cities_graph` should create a `nx.Graph` by cycling through `graph.edges` (from 1.) and for each edge passing the tuple `name1, name2, weights` as arg to it
        1. and replacing `nameX` with a City from 3.2
        1. TIP: you need to pass `data=True` when iterating though nodes, so that you get a dict
    1. create a var `cities_graph` with it
4. Explore by creating a couple of helpers and using them to get a list of neighbours ordered by distance
    1. cities_graph[cities_map["london"]] returns a dictionary, where the key is a `City()` instance, and the value a dictionary in form `{'distance': '53', 'label': '53'})`. They are ordered by insertion order
    1. Create a function `sort_by` which takes each item in the dictionary and sorts it by passing the value to a callable (passed as one of the argument)
    1. Create another function, `by_distance`, which is the callable we shall call this time
        1. Cast the return value to be sure
    1. Should give a list of neightbours arranaged by distance



In [7]:
roadmap_file = "004_roadmap.dot"


# class City(NamedTuple):
#     name: str
#     country: str
#     year: int | None
#     latitude: float
#     longitude: float

#     @classmethod
#     def from_dict(cls, attrs:dict[str, Any]) -> "City":
#         2

3.1
# def as_cities_map(
#         agraph: nx.Graph,
#         node_factory: Callable[[dict[str, Any]], Any]
#     ) -> dict[str, Any]:
#     ...

3.3
# def as_cities_graph(cities_map:dict[str, City], agraph: nx.Graph) -> nx.Graph:
#     ...
4.2
# def sort_by(neighbors: dict[str, City], strategy: Callable[[dict[str, Any]], Any]):
#     return sorted(..., key=lambda item: ...)

4.3
# def by_distance(weights: dict[str, Any]):
#     ...
4.4
# for neighbor, weights in sort_by(cities_graph[cities_map["london"]], by_distance):
#     weights["distance"], neighbor.name

# solution

1
graph = nx.nx_agraph.read_dot(roadmap_file)
graph.nodes["london"]

2
class City(NamedTuple):
    name: str
    country: str
    year: int | None
    latitude: float
    longitude: float

    @classmethod
    def from_dict(cls, attrs:dict[str, Any]) -> "City":
        return cls(
            name=attrs["xlabel"],
            country=attrs["country"],
            year=int(attrs["year"]) or None,
            latitude=float(attrs["latitude"]),
            longitude=float(attrs["longitude"]),
        )

3.1
def as_cities_map(
        agraph: nx.Graph,
        node_factory: Callable[[dict[str, Any]], Any]
    ) -> dict[str, Any]:
    return {
        name: node_factory(attributes)
        for name, attributes in agraph.nodes(data=True)
    }
cities_map = as_cities_map(graph, City.from_dict)

3.3
def as_cities_graph(cities_map:dict[str, City], agraph: nx.Graph) -> nx.Graph:
    return nx.Graph(
        (cities_map[name1], cities_map[name2], weights)
        for name1, name2, weights in agraph.edges(data=True)
    )
cities_graph = as_cities_graph(cities_map, graph)

4.2
def sort_by(neighbors: dict[str, City], strategy: Callable[[dict[str, Any]], Any]):
    return sorted(neighbors.items(), key=lambda item: strategy(item[1]))

4.3
def by_distance(weights: dict[str, Any]):
    return float(weights["distance"])

4.4
for neighbor, weights in sort_by(cities_graph[cities_map["london"]], by_distance):
    weights["distance"], neighbor.name

4.2

4.4

1

{'country': 'England',
 'latitude': '51.507222',
 'longitude': '-0.1275',
 'pos': '80,21!',
 'xlabel': 'City of London',
 'year': '0'}

2

...

('79', 'Southampton')

('85', 'Peterborough')

('100', 'Coventry')

('115', 'Bath')

('118', 'Bristol')

### 001.006 Breadth-First Search Using a FIFO Queue

SPLIT HERE

In the breadth-first search algorithm, you look for a node that satisfies a particular condition by exploring the graph in concentric layers or levels. The breadth-first search algorithm ensures that you’ll eventually explore all connected nodes in a graph while searching for one that satisfies the desired condition. You start traversing the graph at an arbitrarily chosen source node unless the graph is a tree data structure, in which case you typically start at the root node of that tree. At each step, you visit all immediate neighbors of the current node before going deeper. 

1. Reuse `as_cities_map` and `as_cities_graph`
2. nx has a breadth-first search function, `nx.bfs_tree`. Use it to identify the closest neighbour of `CITY` which was granted city status in the XXth century
    1. Use `print_name` to show what the loop is doing
    1. Use the `is_twentieth_century` to find the city and print 
3. The actual order of the neighbours is unknown unless you specify. Let's do that
    1. function `order` does the orderinging
        1. it returns an iterable...
        1. ...and orders by latitude (could be anything really)
    1. add that ordering to the `nx.bfs_tree` function
4. Use the Queue from earlier to implement your own BFS
    1. Create `breadth_first_traverse` which does breadth-first traverse of QUEUE
        1. Initialise the FIFO queue with source
        1. To ensure no infinite loops, keep track of visited nodes and skip them
        1. `breadth_first_traverse` is an iterator that returns the latest dequeued node
            1. (HINT: walrus operator)
        1. once a node is dealt with, add its neightbours to the queue
    1. Create `breadth_first_search` which iterates through `breadth_first_traverse` and returns the first node that matches
    1. call `breadth_first_search` with the same parameters as the `nx.bfs_tree` version and check that results match
5. Add the `order_by` clause, same as `nx.bfs_tree` example, and call function again using `order` for it. It should match previous example
    1. Note that the order_by function is slightly different - it takes a single City, as we are using simple list sorting


In [8]:

# from earlier
1
cities_map = as_cities_map(graph, City.from_dict)
cities_graph = as_cities_graph(cities_map, graph)

CITY = "edinburgh"

def print_name(node):
    print("📍", node.name)

def print_found(node):
    print("Found:", node.name, node.year)

def is_twentieth_century(node):
    return node.year and 1901 <= node.year <= 2000

# 3.1
# def order(neighbors):
#     def by_latitude(city):
#         return city.latitude
#     return ...(sorted(neighbors, key=..., reverse=True))

2, 3
# for node in nx.bfs_tree(G=cities_graph, source=cities_map[CITY]):
#     print_name(node)
#     if is_twentieth_century(node):
#         print_found(node)
#         break
# else:
#     print("Not found")

# 4.1
# def breadth_first_traverse(graph, source):
#     queue = 4.1.1
#     visited = 4.1.2
#     while queue:
#         4.1.3
#         for neighbor in graph.neighbors(node):
#             4.1.4

# 4.2
# def breadth_first_search(graph, source, predicate):
#     ...

# 4.3
# breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century)

5
# def order_by(neighbor: City) -> float:
#     ...

# def breadth_first_traverse(graph, source, order_by=None):
#     ...

# def breadth_first_search(graph, source, order_by=None):
#     ...

# breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century, order)

# solution

2
for node in nx.bfs_tree(G=cities_graph, source=cities_map[CITY]):
    print_name(node)
    if is_twentieth_century(node):
        print_found(node)
        break
else:
    print("Not found")

3
# 3.1
def sort_neighbors(neighbors:list[City]):
    def by_latitude(city):
        return city.latitude
    return iter(sorted(neighbors, key=by_latitude, reverse=True))

3.2
for node in nx.bfs_tree(G=cities_graph, source=cities_map[CITY], sort_neighbors=sort_neighbors):
    print_name(node)
    if is_twentieth_century(node):
        print_found(node)
        break
else:
    print("Not found")

4

def breadth_first_traverse(graph, source):
    queue = Queue(source)
    visited = {source}
    while queue:
        yield (node := queue.dequeue())
        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)

def breadth_first_search(graph, source, predicate):
    for node in breadth_first_traverse(graph, source):
        if predicate(node):
            return node

breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century)

5
def order_by(neighbor: City) -> float:
    return -neighbor.latitude

def breadth_first_traverse_2(graph:nx.Graph, source:City, order_by=None):
    queue = Queue(source)
    visited = {source}
    while queue:
        yield (node := queue.dequeue())
        neighbors:list[City] = list(graph.neighbors(node))
        if order_by:
            neighbors.sort(key=order_by)
        for neighbor in neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)

def breadth_first_search_2(graph:nx.Graph, source:City, predicate, order_by=None):
    for node in breadth_first_traverse_2(graph, source, order_by):
        if predicate(node):
            return node

breadth_first_search_2(cities_graph, cities_map[CITY], is_twentieth_century, order_by)


1

(2, 3)

5

2

📍 Edinburgh
📍 Dundee
📍 Glasgow
📍 Perth
📍 Stirling
📍 Carlisle
📍 Newcastle upon Tyne
📍 Aberdeen
📍 Inverness
📍 Lancaster
Found: Lancaster 1937


...

📍 Edinburgh
📍 Dundee
📍 Perth
📍 Stirling
📍 Glasgow
📍 Newcastle upon Tyne
📍 Carlisle
📍 Aberdeen
📍 Inverness
📍 Sunderland
Found: Sunderland 1992


4

City(name='Lancaster', country='England', year=1937, latitude=54.047, longitude=-2.801)

5

City(name='Sunderland', country='England', year=1992, latitude=54.91, longitude=-1.385)

### 001.007 Shortest Path Using Breadth-First Traversal

In many cases, the fewer the nodes on the path from source to destination, the shorter the distance. Traversing the graph using the breadth-first approach will produce a path guaranteed to have the fewest nodes.

1. Use networkx to reveal all the shortest paths between two cities, which will have the same minimal length
    1. Import cities_map and cities_graph as earlier
    1. Create two cities nodes for aberdeen and perth
    1. nx has a builtin method for that, use it. Start enumerator from 1 and not the default 0
1. Write your own version, `shortest_path` (which only finds the first shortest path)
    1. Base it on `breadth_first_traverse_2`, but keep a dictionary of previous nodes
        1. Keep a trail of nodes in `previous_nodes`
        1. When you find the destination, retrace the trail with a new function `retrace` (next step)
    1. `retrace` takes a dictionary of City -> City and returns a path through them
        1. Store the results in a standard Deque, no need for more
        1. load the destination in a var `current`, and loop through `previous_nodes` to find the trail.
        1. Unshift into the deque as you find it
        1. Ends when the `current` is either None or the source
    1. Call the function, should be the same as one of the paths
        1. Note that you get a different result if you order nodes differently
1. `shortest_path` can also be used to find out whether two nodes are connected or not


In [40]:
# from earlier
# 1.1
cities_map = as_cities_map(graph, City.from_dict)
cities_graph = as_cities_graph(cities_map, graph)

# 1.2
city1 = cities_map["aberdeen"]
city2 = cities_map["perth"]

# 1.3
# for i, path in enumerate(n.all_shortest_paths..., ...):
#     print(f"{i}.", " → ".join(city.name for city in path))

# 2.1
# def shortest_path(
#         graph:nx.Graph, source:City, destination:City, order_by=None
#     ) -> list[City]:
#     queue = Queue(source)
#     visited = {source}
#     previous_nodes = ...
#     while queue:
#         node = queue.dequeue()
#         neighbors:list[City] = list(graph.neighbors(node))
#         if order_by:
#             neighbors.sort(key=order_by)
#         for neighbor in neighbors:
#             if neighbor not in visited:
#                 visited.add(neighbor)
#                 queue.enqueue(neighbor)
#                 # ... 2.1.1
#                 # ...
#                     return retrace(previous_nodes, source, destination)\
#     return []

# 2.2
def retrace(
    previous_nodes:dict[City, City], source:City, destination:City
    ) -> list[City]:
    return []

# 2.3
def by_latitude(city:City):
    return -city.latitude
# " → ".join(city.name for city in shortest_path(cities_graph, city1, city2))
# " → ".join(city.name for city in shortest_path(cities_graph, city1, city2, by_latitude))

# 3
def connected(graph:nx.Graph, source:City, destination:City) -> bool:
    return True

# assert connected(cities_graph, cities_map["belfast"], cities_map["glasgow"]) == False
# assert connected(cities_graph, cities_map["belfast"], cities_map["derry"]) == True

# solution

1.3
for i, path in enumerate(nx.all_shortest_paths(cities_graph, city1, city2), 1):
    print(f"{i}.", " → ".join(city.name for city in path))

2.1
def shortest_path(
        graph:nx.Graph, source:City, destination:City, order_by=None
    ) -> list[City]:
    queue = Queue(source)
    visited = {source}
    previous_nodes = {}
    while queue:
        node = queue.dequeue()
        neighbors:list[City] = list(graph.neighbors(node))
        if order_by:
            neighbors.sort(key=order_by)
        for neighbor in neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)
                previous_nodes[neighbor] = node
                if neighbor == destination:
                    return retrace(previous_nodes, source, destination)
    return []

2.2
def retrace( # type: ignore[no-redef]
    previous_nodes:dict[City, City], source:City, destination:City
    ) -> list[City]:
    path:Deque[City] = deque()

    current: City | None = destination
    while current and current != source:
        path.appendleft(current)
        current = previous_nodes.get(current)
        if current is None:
            return []

    path.appendleft(source)
    return list(path)

2.3
" → ".join(city.name for city in shortest_path(cities_graph, city1, city2))
" → ".join(city.name for city in shortest_path(cities_graph, city1, city2, by_latitude))


# 3
def connected(graph:nx.Graph, source:City, destination:City) -> bool: # type: ignore[no-redef]
    return shortest_path(graph, source, destination) != []

assert connected(cities_graph, cities_map["belfast"], cities_map["glasgow"]) == False
assert connected(cities_graph, cities_map["belfast"], cities_map["derry"]) == True

1.3

1. Aberdeen → Dundee → Perth
2. Aberdeen → Inverness → Perth


2.1

2.2

2.3

'Aberdeen → Dundee → Perth'

'Aberdeen → Inverness → Perth'

### 001.008 Depth-First Search Using a LIFO Queue

The depth-first traversal follows a path from the source node by plunging into the graph as deeply as possible before backtracking to the last edge crossing and trying another branch

1. You'll be reusing `cities_map`, `is_twentieth_century`, etc from earlier
1. Use the built-in method from nx for it, same loop as before
1. `search` is a generic function that takes any traversal function and runs it, it will be used later
1. Write your own `depth_first_traverse`, loosely based on `breadth_first_traverse` but
    1. Use a stack instead of a Queue
    1. What is `visited` inited with? 
    1. There is a condition associated with yielding the node...
    1. The order of when things are added to `visited` is different
    1. Use `search` to test it works
1. You can use python's own call stack instead of creating your own, by writing a recursive version of `depth_first_traverse`.
    1. Use `search` to test it works



In [47]:

# from earlier

CITY = "edinburgh"

2
# for node in nx...
#     print_name(node)
#     if is_twentieth_century(node):
#         print_found(node)
#         break
# else:
#     print("Not found")


3
def search(traverse, graph:nx.Graph, source:City, predicate, order_by=None):
    for node in traverse(graph, source, order_by):
        if predicate(node):
            return node

3
# def depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
#     stack:Stack[City] = Stack(source)
#     visited = ... 3.2
#     while stack:
#         if (node := stack.dequeue()) ...3.3:
#             yield node
#             3.4
#             neighbors = list(graph.neighbors(node))
#             if order_by:
#                 neighbors.sort(key=order_by)
#             for neighbor in reversed(neighbors):
#                 3.4
# 3.5
# search(depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

4
# def recursive_depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
#     visited = set()

#     def visit(node):
#         yield node
#         ...

#     return visit(source)

# 4.1
# search(recursive_depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

# solution

2
for node in nx.dfs_tree(G=cities_graph, source=cities_map[CITY]):
    print_name(node)
    if is_twentieth_century(node):
        print_found(node)
        break
else:
    print("Not found")

3
def depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
    stack:Stack[City] = Stack(source)
    visited = set()
    while stack:
        if (node := stack.dequeue()) not in visited:
            yield node
            visited.add(node)
            neighbors = list(graph.neighbors(node))
            if order_by:
                neighbors.sort(key=order_by)
            for neighbor in reversed(neighbors):
                stack.enqueue(neighbor)

3.5
search(depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

4
def recursive_depth_first_traverse(graph:nx.Graph, source:City, order_by=None):
    visited = set()

    def visit(node):
        yield node
        visited.add(node)
        neighbors = list(graph.neighbors(node))
        if order_by:
            neighbors.sort(key=order_by)
        for neighbor in neighbors:
            if neighbor not in visited:
                yield from visit(neighbor)

    return visit(source)

4.1
search(recursive_depth_first_traverse, cities_graph, cities_map[CITY], is_twentieth_century)

# def breadth_first_search(graph, source, predicate):
#     for node in breadth_first_traverse(graph, source):
#         if predicate(node):
#             return node

# breadth_first_search(cities_graph, cities_map[CITY], is_twentieth_century)

# 5
# def order_by(neighbor: City) -> float:
#     return -neighbor.latitude

# def breadth_first_traverse_2(graph:nx.Graph, source:City, order_by=None):
#     queue = Queue(source)
#     visited = {source}
#     while queue:
#         yield (node := queue.dequeue())
#         neighbors:list[City] = list(graph.neighbors(node))
#         if order_by:
#             neighbors.sort(key=order_by)
#         for neighbor in neighbors:
#             if neighbor not in visited:
#                 visited.add(neighbor)
#                 queue.enqueue(neighbor)

# def breadth_first_search_2(graph:nx.Graph, source:City, predicate, order_by=None):
#     for node in breadth_first_traverse_2(graph, source, order_by):
#         if predicate(node):
#             return node

# breadth_first_search_2(cities_graph, cities_map[CITY], is_twentieth_century, order_by)


2

3

3

5

2

...

3.5

City(name='Lancaster', country='England', year=1937, latitude=54.047, longitude=-2.801)

4

4.1

City(name='Lancaster', country='England', year=1937, latitude=54.047, longitude=-2.801)